XGBoost

In [1]:
import xgboost as xgb
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')

I want to use import_data function from another directory, and make the data ready for the model. LightGBM accepts categorical values, but they have to be encoded as no-negative integers.


In [2]:
from sklearn.model_selection import train_test_split
from display_data import import_data

#We dont want our model to care about the id of the house or the seller
data, data_test = import_data()
Y = data.price
data = data.drop(columns=['price','id','seller'])
categorical_data = ['layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating']
data = data.drop(columns = categorical_data)
data_test = data_test.drop(columns = categorical_data)

for column in data.columns:
    column_type = data[column].dtype
    if column_type == 'object':
        break
    data[column] = data[column].replace(np.NaN, data[column].mean())

X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)


def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

#data.info()
#print(len(Y))
#print(data.corr())

XGB_model = xgb.XGBRegressor(n_estimators = 10000)

XGB_model.fit(X_train,y_train)
prediction = XGB_model.predict(X_test)

##############




#lgb.plot_importance(lightGBM_model)

count = 0
for i in prediction:
    if i < 0:
        prediction[count] = prediction.mean()
    count += 1

rmsle = root_mean_squared_log_error(y_test,prediction)
print("first run", rmsle)


"""
lightGBM_model = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,8,9,10),
    num_leaves=256,
    max_depth=8,
    random_state=42,
    metric='rmsle',
    num_iterations=2000,
    learning_rate=0.09,
    bagging_freq = 5,
    bagging_fraction = 0.8)
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23285 entries, 0 to 23284
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area_total         23285 non-null  float64
 1   area_kitchen       23285 non-null  float64
 2   area_living        23285 non-null  float64
 3   floor              23285 non-null  float64
 4   rooms              23285 non-null  float64
 5   ceiling            23285 non-null  float64
 6   bathrooms_shared   23285 non-null  float64
 7   bathrooms_private  23285 non-null  float64
 8   balconies          23285 non-null  float64
 9   loggias            23285 non-null  float64
 10  phones             23285 non-null  float64
 11  latitude           23285 non-null  float64
 12  longitude          23285 non-null  float64
 13  constructed        23285 non-null  float64
 14  stories            23285 non-null  float64
dtypes: float64(15)
memory usage: 2.7 MB
23285
                   area_tota

"\nlightGBM_model = lgb.LGBMRegressor(\n    categorical_feature=(0,1,2,8,9,10),\n    num_leaves=256,\n    max_depth=8,\n    random_state=42,\n    metric='rmsle',\n    num_iterations=2000,\n    learning_rate=0.09,\n    bagging_freq = 5,\n    bagging_fraction = 0.8)\n"